In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Use BigQuery DataFrames to cluster and characterize complaints

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

The goal of this notebook is to demonstrate a comment characterization algorithm for an online business. We will accomplish this using [Google's PaLM 2](https://ai.google/discover/palm2/) and [KMeans clustering](https://en.wikipedia.org/wiki/K-means_clustering) in three steps:

1. Use PaLM2TextEmbeddingGenerator to [generate text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) for each of 10000 complaints sent to an online bank. If you're not familiar with what a text embedding is, it's a list of numbers that are like coordinates in an imaginary "meaning space" for sentences. (It's like [word embeddings](https://en.wikipedia.org/wiki/Word_embedding), but for more general text.) The important point for our purposes is that similar sentences are close to each other in this imaginary space.
2. Use KMeans clustering to group together complaints whose text embeddings are near to eachother. This will give us sets of similar complaints, but we don't yet know _why_ these complaints are similar.
3. Prompt PaLM2TextGenerator in English asking what the difference is between the groups of complaints that we got. Thanks to the power of modern LLMs, the response might give us a very good idea of what these complaints are all about, but remember to ["understand the limits of your dataset and model."](https://ai.google/responsibility/responsible-ai-practices/#:~:text=Understand%20the%20limitations%20of%20your%20dataset%20and%20model)

We will tie these pieces together in Python using BigQuery DataFrames. [Click here](https://cloud.google.com/bigquery/docs/dataframes-quickstart) to learn more about BigQuery DataFrames!

### Dataset

This notebook uses the [CFPB Consumer Complaint Database](https://console.cloud.google.com/marketplace/product/cfpb/complaint-database).

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (compute)
* BigQuery ML
* Generative AI support on Vertex AI

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models), [Generative AI support on Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing#generative_ai_models),
and [BigQuery ML pricing](https://cloud.google.com/bigquery/pricing#bqml),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

Complete the tasks in this section to set up your environment.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,aiplatform.googleapis.com) to enable the following APIs:

  * BigQuery API
  * BigQuery Connection API
  * Vertex AI API

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, see the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
# set your project ID below
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id in gcloud
! gcloud config set project {PROJECT_ID}

#### Set the region

You can also change the `REGION` variable used by BigQuery. Learn more about [BigQuery regions](https://cloud.google.com/bigquery/docs/locations#supported_locations).

In [3]:
REGION = "US"  # @param {type: "string"}

#### Authenticate your Google Cloud account

Depending on your Jupyter environment, you might have to manually authenticate. Follow the relevant instructions below.

**Vertex AI Workbench**

Do nothing, you are already authenticated.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [4]:
# ! gcloud auth login

**Colab**

Uncomment and run the following cell:

In [5]:
# from google.colab import auth
# auth.authenticate_user()

Now we are ready to use BigQuery DataFrames!

## Step 1: Text embedding 

BigQuery DataFrames setup

In [6]:
import bigframes.pandas as bf

bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

If you want to reset the location of the created DataFrame or Series objects, reset the session by executing `bf.close_session()`. After that, you can reuse `bf.options.bigquery.location` to specify another location.

Data Input - read the data from a publicly available BigQuery dataset

In [7]:
input_df = bf.read_gbq("bigquery-public-data.cfpb_complaints.complaint_database")

In [8]:
issues_df = input_df[["consumer_complaint_narrative"]].dropna()
issues_df.head(n=5) # View the first five complaints

,consumer_complaint_narrative
2,COLLECTION BUREAU OF AMERICA ACCOUNT NO. XXXX...
3,"Despite multiple written requests, the unverif..."
6,Once again you guys have not provided me with ...
9,XX/XX/XXXX {$350.00} I received a outstating d...
10,Im am unable to withdraw money from my account...


Downsample DataFrame to 10,000 records for model training.

In [9]:
# Choose 10,000 complaints randomly and store them in a column in a DataFrame
downsampled_issues_df = issues_df.sample(n=10000)

Generate the text embeddings

In [10]:
from bigframes.ml.llm import PaLM2TextEmbeddingGenerator

model = PaLM2TextEmbeddingGenerator() # No connection id needed

In [11]:
# Will take ~3 minutes to compute the embeddings
predicted_embeddings = model.predict(downsampled_issues_df)
# Notice the lists of numbers that are our text embeddings for each complaint
predicted_embeddings.head() 

,text_embedding,statistics,ml_embed_text_status,content
3,"[0.02506784163415432, -0.03947214409708977, -0...","{""token_count"":273,""truncated"":false}",,"Despite multiple written requests, the unverif..."
102,"[0.029197776690125465, -0.028653817251324654, ...","{""token_count"":859,""truncated"":false}",,"To Whom It May Concern, Please be advised that..."
660,"[0.034257132560014725, -0.05359702184796333, -...","{""token_count"":282,""truncated"":false}",,Transunion has failed to remove a XXXX XXXX b...
1500,"[-0.008016454987227917, -0.05421802029013634, ...","{""token_count"":39,""truncated"":false}",,FRAUDULENT TRADELINES WERE ATTRIBUTED TO MY CR...
1601,"[-0.005403461866080761, -0.024799197912216187,...","{""token_count"":94,""truncated"":false}",,I am a victim of identity theft/fraud. I have ...


We now have the complaints and their text embeddings as two columns in our predicted_embeddings DataFrame.

## Step 2: Create k-means model and predict clusters

In [12]:
from bigframes.ml.cluster import KMeans

cluster_model = KMeans(n_clusters=10) # We will divide our complaints into 10 groups

Perform KMeans clustering

In [13]:
# Use KMeans clustering to calculate our groups. Will take ~3 minutes.
cluster_model.fit(predicted_embeddings[["text_embedding"]])
clustered_result = cluster_model.predict(predicted_embeddings)
# Notice the CENTROID_ID column, which is the ID number of the group that
# each complaint belongs to.
clustered_result.head(n=5)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,text_embedding,statistics,ml_embed_text_status,content
3,8,"[{'CENTROID_ID': 8, 'DISTANCE': 0.399505154607...","[0.02506784163415432, -0.03947214409708977, -0...","{""token_count"":273,""truncated"":false}",,"Despite multiple written requests, the unverif..."
102,8,"[{'CENTROID_ID': 8, 'DISTANCE': 0.403332660368...","[0.029197776690125465, -0.028653817251324654, ...","{""token_count"":859,""truncated"":false}",,"To Whom It May Concern, Please be advised that..."
660,10,"[{'CENTROID_ID': 10, 'DISTANCE': 0.42615208239...","[0.034257132560014725, -0.05359702184796333, -...","{""token_count"":282,""truncated"":false}",,Transunion has failed to remove a XXXX XXXX b...
1500,6,"[{'CENTROID_ID': 6, 'DISTANCE': 0.500031509322...","[-0.008016454987227917, -0.05421802029013634, ...","{""token_count"":39,""truncated"":false}",,FRAUDULENT TRADELINES WERE ATTRIBUTED TO MY CR...
1601,6,"[{'CENTROID_ID': 6, 'DISTANCE': 0.361769337067...","[-0.005403461866080761, -0.024799197912216187,...","{""token_count"":94,""truncated"":false}",,I am a victim of identity theft/fraud. I have ...


Our DataFrame clustered_result now has an additional column that includes an ID from 1-10 (inclusive) indicating which semantically similar group they belong to.

## Step 3: Use PaLM2 LLM model to summarize complaint clusters

Build prompts - we will choose just two of our categories and prompt PaLM2TextGenerator to identify their salient characteristics. The prompt is natural language in a python string.

In [14]:
# Using bigframes, with syntax identical to pandas,
# filter out the first and second groups
cluster_1_result = clustered_result[
    clustered_result["CENTROID_ID"] == 1
][["content"]]
cluster_1_result_pandas = cluster_1_result.head(5).to_pandas()

cluster_2_result = clustered_result[
    clustered_result["CENTROID_ID"] == 2
][["content"]]
cluster_2_result_pandas = cluster_2_result.head(5).to_pandas()

In [15]:
# Build plain-text prompts to send to PaLM 2. Use only 5 complaints from each group.
prompt1 = 'comment list 1:\n'
for i in range(5):
    prompt1 += str(i + 1) + '. ' + \
        cluster_1_result_pandas["content"].iloc[i] + '\n'

prompt2 = 'comment list 2:\n'
for i in range(5):
    prompt2 += str(i + 1) + '. ' + \
        cluster_2_result_pandas["content"].iloc[i] + '\n'

print(prompt1)
print(prompt2)

comment list 1:
1. Out of the blue I received a debt collection notice saying they were collecting debt of {$710.00} for a XXXX XXXX  which I have no idea what it was and was never notified of. I replied with written notification disputing the debt and demand of supporting documentation ( as the debt collection simply stated a total due with no supporting documentation ) but received no supporting documentation or verification.
2. I am referring a case to the Consumer Protection Bureau as follows. 

First, please see the most recent document, 'New_Debt_Collector_Account.jpg '. This was sent to me on XX/XX/2021 and arrived on XX/XX/2021. This is a letter from " I.C. System '' telling me I must pay a delinquent account for " XXXX ''. An account I thought that was already settled by " Offices of the XXXX XXXX XXXX ''. Or so I believed. Until now... 

See attachment 'dispute.rtf '. This is the original letter I sent to " Offices of the XXXX XXXX XXXX ''. Within 'dispute.rtf ' is a very det

In [16]:
# The plain English request we will make of PaLM 2
prompt = (
    "Please highlight the most obvious difference between "
    "the two lists of comments:\n" + prompt1 + prompt2
)
print(prompt)

Please highlight the most obvious difference between the two lists of comments:
comment list 1:
1. Out of the blue I received a debt collection notice saying they were collecting debt of {$710.00} for a XXXX XXXX  which I have no idea what it was and was never notified of. I replied with written notification disputing the debt and demand of supporting documentation ( as the debt collection simply stated a total due with no supporting documentation ) but received no supporting documentation or verification.
2. I am referring a case to the Consumer Protection Bureau as follows. 

First, please see the most recent document, 'New_Debt_Collector_Account.jpg '. This was sent to me on XX/XX/2021 and arrived on XX/XX/2021. This is a letter from " I.C. System '' telling me I must pay a delinquent account for " XXXX ''. An account I thought that was already settled by " Offices of the XXXX XXXX XXXX ''. Or so I believed. Until now... 

See attachment 'dispute.rtf '. This is the original letter I

Get a response from PaLM 2 LLM by making a call to Vertex AI using our connection.

In [17]:
from bigframes.ml.llm import PaLM2TextGenerator

q_a_model = PaLM2TextGenerator()

In [18]:
# Make a DataFrame containing only a single row with our prompt for PaLM 2
df = bf.DataFrame({"prompt": [prompt]})

In [19]:
# Send the request for PaLM 2 to generate a response to our prompt
major_difference = q_a_model.predict(df)
# PaLM 2's response is the only row in the dataframe result 
major_difference["ml_generate_text_llm_result"].iloc[0]

' The most obvious difference between the two lists of comments is the nature of the issues being raised.\n\n**Comment list 1 primarily focuses on disputes related to debt collection practices.** \n\n- Commenters express concerns about receiving debt collection notices for debts they believe they do not owe, lack of supporting documentation, and the transfer of debt between debt collection agencies without proper resolution. \n\n- They highlight the impact on their credit scores and the frustration of dealing with misleading or deceptive practices.\n\n\n**Comment list 2, on the other hand, covers a wider range of issues related to financial services.** \n\n- It includes complaints about fraudulent charges'

We now see PaLM2TextGenerator's characterization of the different comment groups. Thanks for using BigQuery DataFrames!

# Summary and next steps

You've used the ML and LLM capabilities of BigQuery DataFrames to help analyze and understand a large dataset of unstructured feedback.

Learn more about BigQuery DataFrames in the [documentation](https://cloud.google.com/python/docs/reference/bigframes/latest) and find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks).